## Домашнее задание, реализация логрега

In [31]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import datasets
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [18]:
# Загружаем набор данных Ирисы:
iris = datasets.load_iris()

data = pd.DataFrame(data= np.c_[iris['data'], iris['target']],
                     columns= iris['feature_names'] + ['target'])


data['target'] = pd.Categorical.from_codes(iris.target, iris.target_names)

# Убираем ненужный класс в таргете и сбрасываем индекс:
data = data[data['target']!='setosa'].reset_index().drop('index', axis=1)

data.sample(5)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
134,6.1,2.6,5.6,1.4,virginica
119,6.0,2.2,5.0,1.5,virginica
63,6.1,2.9,4.7,1.4,versicolor
133,6.3,2.8,5.1,1.5,virginica
46,5.1,3.8,1.6,0.2,setosa


In [23]:
from sklearn.preprocessing import LabelEncoder

#Закодируем таргет обратно

label_encoder = LabelEncoder()
data['target'] = label_encoder.fit_transform(data['target'])
data.sample(5)

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
3,5.5,2.3,4.0,1.3,0
79,7.2,3.0,5.8,1.6,1
25,6.6,3.0,4.4,1.4,0
91,6.9,3.1,5.1,2.3,1
21,6.1,2.8,4.0,1.3,0


In [133]:
X = data.loc[:,data.columns[:4]]
y = data.loc[:,data.columns[-1:]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [198]:
class LogisticRegression:
    def __init__(self, penalty="l2", gamma=0, fit_intercept=True, self_moment='grad'):
        r"""
        A simple logistic regression model fit via gradient descent on the
        penalized negative log likelihood.

        Notes
        -----
        For logistic regression, the penalized negative log likelihood of the
        targets **y** under the current model is

        .. math::

            - \log \mathcal{L}(\mathbf{b}, \mathbf{y}) = -\frac{1}{N} \left[
                \left(
                    \sum_{i=0}^N y_i \log(\hat{y}_i) +
                      (1-y_i) \log(1-\hat{y}_i)
                \right) - R(\mathbf{b}, \gamma) 
            \right]
        
        where
        
        .. math::
        
            R(\mathbf{b}, \gamma) = \left\{
                \begin{array}{lr}
                    \frac{\gamma}{2} ||\mathbf{beta}||_2^2 & :\texttt{ penalty = 'l2'}\\
                    \gamma ||\beta||_1 & :\texttt{ penalty = 'l1'}
                \end{array}
                \right.
                
        is a regularization penalty, :math:`\gamma` is a regularization weight, 
        `N` is the number of examples in **y**, and **b** is the vector of model 
        coefficients.

        Parameters
        ----------
        penalty : {'l1', 'l2'}
            The type of regularization penalty to apply on the coefficients
            `beta`. Default is 'l2'.
        gamma : float
            The regularization weight. Larger values correspond to larger
            regularization penalties, and a value of 0 indicates no penalty.
            Default is 0.
        fit_intercept : bool
            Whether to fit an intercept term in addition to the coefficients in
            b. If True, the estimates for `beta` will have `M + 1` dimensions,
            where the first dimension corresponds to the intercept. Default is
            True.
        self_moment : char
            grad or nesterov_momentum or RMSProp
        """
        err_msg = "penalty must be 'l1' or 'l2', but got: {}".format(penalty)
        assert penalty in ["l2", "l1"], err_msg
        self.beta = None
        self.gamma = gamma
        self.penalty = penalty
        self.fit_intercept = fit_intercept
        self.moment = self_moment


    def fit(self, X, y, lr=0.01, tol=1e-7, max_iter=1e6, nesterov_coef=0.7):
        """
        Fit the regression coefficients via gradient descent on the negative
        log likelihood.

        Parameters
        ----------
        X : :py:class:`ndarray <numpy.ndarray>` of shape `(N, M)`
            A dataset consisting of `N` examples, each of dimension `M`.
        y : :py:class:`ndarray <numpy.ndarray>` of shape `(N,)`
            The binary targets for each of the `N` examples in `X`.
        lr : float
            The gradient descent learning rate. Default is 1e-7.
        max_iter : float
            The maximum number of iterations to run the gradient descent
            solver. Default is 1e7.
        nesterov_coef : float
            The number of nesterov_coef. Default is 0.8.
        """
        # convert X to a design matrix if we're fitting an intercept
        if self.fit_intercept:
            X = np.c_[np.ones(X.shape[0]), X]

        l_prev = np.inf
        loss_hist = []
        grad_prev = 0
        EG = 0 
        self.beta = np.random.rand(X.shape[1])
        for _ in range(int(max_iter)):
            y_pred = sigmoid(np.dot(X, self.beta))
            loss = self._NLL(X, y, y_pred)
            loss_hist.append(loss)
            if l_prev - loss < tol:
                return
            l_prev = loss
            if self.moment == 'grad':
                self.beta -= lr * self._NLL_grad(X, y, y_pred)
                
            elif self.moment == 'nesterov_momentum':
                self.beta -= (nesterov_coef * grad_prev + (1 - nesterov_coef) * lr * self._NLL_grad(X, y, y_pred))
                grad_prev = nesterov_coef * grad_prev + (1 - nesterov_coef) * lr * self._NLL_grad(X, y, y_pred)
            elif self.moment == 'RMSProp':
                EG = nesterov_coef * EG + (1 - nesterov_coef)*self._NLL_grad(X, y, y_pred)*self._NLL_grad(X, y, y_pred)
                self.beta -=   lr / np.sqrt(EG+tol) * self._NLL_grad(X, y, y_pred)
                

    def _NLL(self, X, y, y_pred):
        r"""
        Penalized negative log likelihood of the targets under the current
        model.

        .. math::

            \text{NLL} = -\frac{1}{N} \left[
                \left(
                    \sum_{i=0}^N y_i \log(\hat{y}_i) + (1-y_i) \log(1-\hat{y}_i)
                \right) - R(\mathbf{b}, \gamma)
            \right]
        """
        N, M = X.shape
        beta, gamma = self.beta, self.gamma 
        order = 2 if self.penalty == "l2" else 1
        norm_beta = np.linalg.norm(beta, ord=order)
        
        nll = -np.log(y_pred[y.iloc[:,0] == 1]).sum() - np.log(1 - y_pred[y.iloc[:,0] == 0]).sum()
        penalty = (gamma / 2) * norm_beta ** 2 if order == 2 else gamma * norm_beta
        return (penalty + nll) / N

    def _NLL_grad(self, X, y, y_pred):
        """Gradient of the penalized negative log likelihood wrt beta"""
        N, M = X.shape
        l1norm = lambda x: np.linalg.norm(x, 1)  # noqa: E731
        p, beta, gamma = self.penalty, self.beta, self.gamma
        d_penalty = gamma * beta if p == "l2" else gamma * np.sign(beta)
        return -(np.dot(y.iloc[:,0] - y_pred, X) + d_penalty) / N
   


    def predict(self, X):
        """
        Use the trained model to generate prediction probabilities on a new
        collection of data points.

        Parameters
        ----------
        X : :py:class:`ndarray <numpy.ndarray>` of shape `(Z, M)`
            A dataset consisting of `Z` new examples, each of dimension `M`.

        Returns
        -------
        y_pred : :py:class:`ndarray <numpy.ndarray>` of shape `(Z,)`
            The model prediction probabilities for the items in `X`.
        """
        # convert X to a design matrix if we're fitting an intercept
        if self.fit_intercept:
            X = np.c_[np.ones(X.shape[0]), X]
        return sigmoid(np.dot(X, self.beta))
 
 #мои попытки написать нужные функции или скопированные из интернета
def sigmoid(x):
    """The logistic sigmoid function"""
    return 1 / (1 + np.exp(-x))

def log_loss(h, y):
    loss = (-y * np.log(h) - (1 - y) * np.log(1 - h)).mean()
    return loss
def gradient_descent(X, y, w, lr, num_iter):
    losses = []
    for i in range(num_iter):
        z = np.dot(X, w)
        h = sigmoid(z)
        gradient = np.dot(X.T, (h -y)) / len(y)
        w -= lr * gradient
        loss = log_loss(h, y)
        losses.append(loss)
    
    return w, losses

def RMSProp(X, gamma, lr=0.25, eps=0.00001):
    Y = []
    EG = 0
    for x in X:
        EG = gamma*EG + (1-gamma)*x*x
        v = lr/np.sqrt(EG + eps)*x
        Y.append(v)
    return np.asarray(Y)

## Градиент

In [187]:
%%time
model=LogisticRegression(self_moment='grad')

model.fit(X_train, y_train)

Wall time: 1min 7s


In [188]:
y_pred = model.predict(X_test)

In [189]:
score_grad = model._NLL(X_test, y_test, y_pred )
score_grad

0.26678719815247637

### Нестеров

In [199]:
model_nesterov=LogisticRegression(self_moment='nesterov_momentum')

Wall time: 0 ns


In [200]:
%%time
model_nesterov.fit(X_train, y_train,nesterov_coef=0.7)

Wall time: 1min 30s


In [192]:
y_pred_nesterov = model_nesterov.predict(X_test)

In [193]:
score_nesterov = model_nesterov._NLL(X_test, y_test, y_pred_nesterov )
score_nesterov

0.269743322993872

### RMSProp

In [194]:
model_RMSProp=LogisticRegression(self_moment='RMSProp')

In [202]:
%%time
model_RMSProp.fit(X_train, y_train)

Wall time: 10.3 s


In [196]:
y_pred_RMSProp = model_RMSProp.predict(X_test)

In [201]:
score_RMSProp = model_RMSProp._NLL(X_test, y_test, y_pred_RMSProp )
score_RMSProp

0.4568677970632627